In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from pandas.plotting import scatter_matrix
from sklearn.ensemble import RandomForestRegressor


import seaborn as sns 
import matplotlib.pyplot as plt
import inspect

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.isna().sum()
df.ocean_proximity.unique()

In [ ]:
total_bedroom = df.total_bedrooms

In [ ]:
print(total_bedroom[total_bedroom.isna()])
# replace total bedrooms NaN values with there median value
median = total_bedroom.median()
print(f"median is :{median}")
total_bedroom.fillna(total_bedroom.median(),inplace = True)
print(f"the NAN values in total_bedroom now is :{total_bedroom.isna().sum()}")
# or total_bedroom.fillna(total_bedroom.median(),inplace = True) to do the same in one line 


In [ ]:

df.plot()


#  first i need to scale the  data , for that either i will use scikit-learn preprocessor or i can scale it by writing 4-5 line code 

In [ ]:


scaler = StandardScaler()

numeric_df = df.copy()
numeric_df.drop(columns =['ocean_proximity'],inplace = True)

# scaled_numeric_data = scaler.fit_transform(numeric_df)
# SINCE THE VALUES FOR HOUSE VALUE WAS COMING NEGATIVE BY USING STANDARD SCALAR METHOD I WILL USE NORMALIZATION 


scaled_numeric_data = scaler.fit_transform(numeric_df)


scaled_numeric_data = pd.DataFrame(scaled_numeric_data,columns = numeric_df.columns)


In [ ]:
corr_matrix  = scaled_numeric_data.corr()
plt.figure(figsize=(10,10))
# plt.imshow(corr_matrix, cmap ='coolwarm',interpolation='nearest') ----- i donot like this plot let's try with sns 

sns.heatmap(corr_matrix , annot=True , cmap= 'coolwarm')
plt.title("Heat_Map_Of_Correlation_Matrix")

In [ ]:
def numeric_dataframe(dataframe):
    numeric_df = dataframe.select_dtypes(include ='number')
    return numeric_df

In [ ]:
def plot_scatter_matrix(numeric_dataframe):
  
    # if i just want to plot some of the attributes then i will make a list of them and will pass it to dataframe in the scatter matrix function 
    # attributes = ['A','B']
    # scatter_matrix(dataframe[attributes],figsize = (10,8))
    # i will plot the correcln matrix for all the attributes present in the DataFrame 
    scatter_matrix(numeric_dataframe,figsize=(10,15),hist_kwds={'bins': 20},alpha =0.5,diagonal='kde')

In [ ]:
numeric_df = numeric_dataframe(df)
# plot_scatter_matrix(numeric_df)

In [ ]:
#now for non-numeric columns i need  one hot encoding 
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output = False)

ocean_proximity  = df[['ocean_proximity']]
# non_numeric = encoder.fit_transform(ocean_proximity)
# non_numeric is a sparse matrix (which stores the location of category =1 , it is efficent way of utilixing 
#memory when dataset is very large , by default sparse = true ,if we do sparse = False then this(variable) will be a dense matrix )
# use OneHotEncoder (sparse_output = False) to get a dense matrix 
non_numeric = encoder.fit_transform(ocean_proximity)
encoding_columns = encoder.get_feature_names_out(['ocean_proximity'])# to get the names of column , we need it when we make DataFrame and concatenate the value with the numeric values 
non_numeric = pd.DataFrame(non_numeric,columns = ['less_than1H_ocean','INLAND','ISLAND','NEAR_BAY','NEAR_OCEAN'])


In [ ]:
new_df = pd.concat([non_numeric,scaled_numeric_data],axis =1)
new_df

In [ ]:
Y = new_df['median_house_value']
print(new_df.columns)
X = new_df.drop(columns = ['median_house_value'])


In [ ]:
new_df.hist(bins =100,figsize=(20,20));

In [ ]:

x_train , x_test , y_train , y_test = train_test_split(X,Y,train_size= 0.7,random_state=13)
# y_train = y_train.values.reshape(-1,1)# to reshape in 2 dimensional array which have 1 columns 
# y_test = y_test.values.reshape(-1,1)
# randomForestClassifer asking for (n,) 1 dimensional array 
x_train.isna().sum()

In [ ]:
x_test,x_dev,y_test,y_dev = train_test_split(x_test,y_test,test_size = 0.5)
print(x_test.shape,x_dev.shape,y_test.shape ,y_dev.shape)

In [ ]:

def model_check_cv(X,Y,Model):
    
    x_train,x_test,y_train, y_test = train_test_split(X,Y,test_size =0.3)
    x_test,x_dev,y_test,y_dev= train_test_split(x_test,y_test,test_size = 0.3)
    print(f" shapes are x_train -{x_train.shape} , y_train-   {y_train.shape}, x_test - {x_test.shape} ,y_test-  {y_test.shape} , x_dev -  {x_dev.shape}   , y_dev -{y_dev.shape}")

    model = Model
   
    scores = cross_val_score(Model,x_train,y_train,scoring = 'neg_mean_squared_error',cv=10)
    rmse_score = np.sqrt(-scores)

    print("scores : ",rmse_score)
    print("mean : ",rmse_score.mean())
    print("standard_deviation : ",rmse_score.std())

In [ ]:
from xgboost import XGBRegressor
model_check_cv(X,Y,XGBRegressor())

In [ ]:
model_check_cv(X,Y,RandomForestRegressor())
print()
print()
model_check_cv(X,Y,LinearRegression())
print()
print()
model_check_cv(X,Y,DecisionTreeRegressor())
print()
print()
model_check_cv(X,Y,SVR())
 

In [ ]:
# grid search for Random Forest
random_forest = RandomForestRegressor()
parameter_grid =[{'n_estimators':[330,350,370,390],
                  'max_depth':[29]}]
grid_search = GridSearchCV(random_forest,parameter_grid,cv=5,scoring = 'neg_mean_squared_error',return_train_score=True)
grid_search.fit(x_dev,y_dev)


In [ ]:
grid_search.best_params_

In [ ]:
# grid serach for Decision Trees
DecisionTreeRegressor()
parameter_grid =[{'max_depth':[10,20,30],
                  'min_samples_leaf':[1,2,10,20,30,40,50,100,800],
                 'max_features':[1,2,5,10,20,30,50,40],
                }]
grid_search = GridSearchCV(DecisionTreeRegressor(),parameter_grid,cv=5,scoring = 'neg_mean_squared_error',return_train_score=True)
grid_search.fit(x_dev,y_dev)

In [ ]:
grid_search.best_params_

In [ ]:
# xg boost regressor hyperparameter 
xg_regressor = XGBRegressor()
parameter_grid =[{'learning_rate':[0.0001,0.00001],
                  'max_depth':[29]}]
grid_search = GridSearchCV(random_forest,parameter_grid,cv=5,scoring = 'neg_mean_squared_error',return_train_score=True)
grid_search.fit(x_dev,y_dev)


In [ ]:
grid_search.best_params_

In [ ]:
####################### i havenot studied SVM so i donot understand its hyperparameters
#grid search for SVM Regressor
# parameter_grid =[{'max_de':[330,350,370,390],
#                   'max_depth':[29]}]
# grid_search = GridSearchCV(SVR(),parameter_grid,cv=5,scoring = 'neg_mean_squared_error',return_train_score=True)
# grid_search.fit(x_dev,y_dev)

In [ ]:
# use this to check the arguments which we can pass for a function

inspect.signature(sns.heatmap)

In [ ]:

model_check(X,Y,LinearRegression(copy_X=True))
    
